# FP management

In thi notebook we will define the various methods to perform changes to the database to keep track of the various events. In particular we will take care of:

- Adding new free agent players to the database, when they are included in the FC list
- Assigning trophies
- Assigning fines
- Calculating total roster value to determine the salary, extra budget and luxury tax
- Update team_real
- TBD
- UPDATE PLAYER NAME IF IT HAS CHANGED ON FC:
    - Sometime FC changes the name of a player. In 2019/2020 LUKAKU was 'LUKAKU R.', now just 'LUKAKU', because Jordan left serie A. We need a way of scanning the players in our db by ID and update the name as needed. Maybe it can be included in the stats update algorithm
    

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo
import progressbar
with open('credential.json','r') as f:
    cred = json.load(f)
stats = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl')

In [2]:
cluster = MongoClient(cred['cred'])
db = cluster['Game']
collection = db['Players']

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))


## Update personal info
this part is to run through the players in our database and cross check the info with the stats db:
- if the player is not in our db, we don't care: we'll add it when someone wants to buy them
- if the player is in our db but not in stats it means that it left serie A, so the algorithm should change the team to none:
```collection.update_one({'_id': pl['_id']},{'$set':{'info.personal_info.team_real': None}})
```
- if the player has changed teams we need to update the information
- update name if it has changed, based on id

In [3]:
def update_personal_info(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]

    player_out = []
    posts = collection.find({})
    stats.index = list(stats['Id'])
    for dic in posts:
        Id = dic['_id']
        if Id in stats.index:
            collection.update_one({'_id': Id},{'$set':{'name': stats.loc[Id].Nome }})
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': stats.loc[Id].Squadra}})
        else:
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': None}})
            player_out.append(collection.find_one({'_id': Id})['name'])
    
    return player_out

In [4]:
#This updates the personal info, name and team, based on the file stats produced in the IGNOBEL notebook.
#the function returns the list of players that are in our db but no longer in FC (e.g. those who retired or left serie a)
update_personal_info(stats)

['EMPEREUR']

## UPDATE stats

In [9]:
def update_stats(stats, Id, CR = cred['cred'], DB = 'Game', CO = 'Players'):

    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    stats.index = list(stats.Id)
    #print(stats[stats['Id'] == int(Id)].T[0][4:20])
    stats_dict = dict(stats[stats['Id'] == int(Id)].T[Id][4:20])

    collection.update_one({'_id': Id}, {'$set': {'info.stats': stats_dict}})

In [10]:
def update_stats_all(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    all_pl = list(collection.find({}))
    
    for pl in progressbar.progressbar(all_pl):
        #print(pl['_id'])
        if len(stats[stats['Id'] == pl['_id']]) == 0:
            print(pl['name']+' left serie A')
            #
            continue
        else:
            update_stats(stats, pl['_id'], CR, DB, CO)

In [5]:
# algorithm runs through the entire db and updates stats value
# if player no more in serie a it prints a message, but we need to include change of team_real to none
#it is probably best to 
update_stats_all(stats)

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))
 65% (368 of 563) |##############        | Elapsed Time: 0:01:19 ETA:   0:02:07

EMPEREURleft serie a, modify algorithm to change team_real to None. profile not in FC db anymore


100% (563 of 563) |######################| Elapsed Time: 0:01:58 Time:  0:01:58


## Add player to database

In [24]:
def add_player(Id, stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    # Dictionary with player current team and loan info
    CurrentTeamDict = {
        'owner': None, #luca, pietro etc
        'squad': None, # 'Main' or 'Primavera'
        'start_date': '',
        'previous_team': None, #last team (e.g. owner, squad)
        'quotation_initial': 0,
        'on_loan': False, #True or False
        'loan_info': None
    }

    # Dictionary with player ownership info
    ContractDict = {
        'owner': None, #this seems redundant
        'start_date': '',
        'cost': 0,
        'acquisition_mode': None, #asta_svincolati, draft, acquisto
        'previous_owner': None, #owner or None
        'quotation_initial': 0
    }

    # Dictionary with player personal info
    PersonalInfoDict = {
        'full_name': '',
        'birthdate': '01/01/1970',
        'nation': '',
        'team_real': '',
        'FC_role': ''
    }

    # Nested dictionary for a single player info
    PlayerDict = {
        '_id': 0,
        'name': '',
        'personal_info': PersonalInfoDict,
        'contract': ContractDict,
        'current_team': CurrentTeamDict,
    }
    transStat = stats.T
    PlayerDict['_id'] = Id
    PlayerDict['name'] = transStat[Id]['Nome']
    PlayerDict['personal_info']['FC_role'] = transStat[Id]['R']
    PlayerDict['personal_info']['team_real'] = transStat[Id]['Squadra']
    PlayerDict['personal_info']['full_name'] = transStat[Id]['Nome Completo']
    PlayerDict['personal_info']['nation'] = transStat[Id]['Nazionalit\'a']
    PlayerDict['personal_info']['birthdate'] = transStat[Id]['Classe']
    PlayerDict['contract']['quotation_initial'] = transStat[Id]['Qt. I']
    PlayerDict['contract']['owner'] = None
    PlayerDict['current_team']['owner'] = None
    PlayerDict['current_team']['quotation_initial'] = transStat[Id]['Qt. I'] 
    temp = dict(transStat[Id][4:20])
    temp['Qt_I'] = temp.pop('Qt. I')
    temp['Qt_A'] = temp.pop('Qt. A')
    PlayerDict['stats'] = temp
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    collection.insert_one(PlayerDict)
    
    return PlayerDict

In [25]:
#this adds a new player to the DB, without assigning them to any team
#this NEEDS TO BE TESTED, with a player who is not in our db
#add_player(Id, stats)

## Manage roster value and extra budget/luxury tax